#### Hydrogen atomic orbitals: Part 2
In this notebook we will introduce 2D and 3D plotting.  Let's start by copying the function for evaluating the hydrogen wavefunction from the previous notebook.

In [ ]:
import numpy as np
import scipy as sp
import scipy.special as sps
import matplotlib.pyplot as plt

def h_orbital(r,theta,phi,n,l,m):
    pf = ( (2./n)**3. * sps.factorial(n-l-1) / (2. * n * sps.factorial(n+l)) )**0.5
    return pf * np.exp(-r/2.) *r**l * sps.sph_harm(m,l,phi,theta) * sps.eval_genlaguerre(n-l-1,2*l+1,r)

#### Conversion of Cartesian to spherical coordinates

Before we get started on making 2D and 3D plots, we need a function that can convert Cartesian coordinates into spherical coordinates.  Using trigonometric functions, it is easy to derive the expressions for the spherical coordinates from $x, y, z$ as follows:

$$ r = \sqrt{x^2 + y^2 + z^2} ; \quad \theta = \cos^{-1} \frac{z}{r} ; \quad\phi = \tan^{-1} \frac{y}{x} $$

However, when applying these formulas there is quickly a problem.  Notice that for $\theta$, there will be a divide-by-zero error at the origin.  Even worse, the formula for $\phi$ returns the same value for $(-x, -y)$ as for $(x, y)$, even though we know the actual $\phi$ angle differs by $\pi$ for these two points.  

Both problems can be solved by using the `np.arctan2(y, x)` function.  This function takes in two arguments that would normally be divided in the conventional arctangent, thus giving it the ability to return values in the full range of $\phi$.  Also, this function returns standard values when one or both inputs are equal to zero; if we had used `np.arctan`, this would have resulted in errors.  

The boardwork below shows how `np.arctan2` is used in place of `np.arccos` and `np.arcsin` to give a robust conversion from Cartesian to spherical coordinates.

<div>
<img src="https://raw.githubusercontent.com/leeping/che155/master/assets/images/week02/cart-sph.jpg" width="400"/>
</div>

In [ ]:
def to_spherical(x, y, z):
    """Converts the Cartesian coordinates x,y,z to r,theta,phi

    This function expects x, y, and z to be 3D arrays,
    such as those generated by np.meshgrid. The output arrays are
    always dense 3D grids, and correspond to r, theta, and phi

    Parameters
    ----------
    x, y, z : np.ndarray
        Cartesian x / y / z values, in a.u.

    Returns
    -------
    tuple of 3x np.ndarrays
        3D grid of r, theta, phi values
    """

    r = ( x**2 + y**2 + z**2 )**0.5
    theta = np.arctan2((x**2 + y**2)**0.5, z)
    phi = np.arctan2(y, x)

    return r, theta, phi

#### Plotting heat maps of orbital slices in 2D

In what follows, we will create a 2-D grid where two out of the three $(x, y, z)$ coordinates vary and the other coordinate is set to a constant.  Note that we still have three input arrays for `to_spherical()` because all three Cartesian coordinates are needed to calculate the corresponding spherical coordinates.  The shape of the 3-D arrays in the following cells is (101, 1, 101).  The spherical coordinates are then used as input to the orbital function.

In [ ]:
# Create range of values along each axis.
x = np.linspace(-10, 10, 101)
y = 0.0
z = np.linspace(-10, 10, 101)

# Create 3D arrays containing x, y, z-coords of each point.
# The 'indexing' keyword argument is needed, otherwise the 
# ordering of our dimensions will be messed up.
xg, yg, zg = np.meshgrid(x, y, z, indexing='ij')

print("The shape of xg is: ", xg.shape)

# Calculate r, theta, and phi values at each grid point.
rg, tg, pg = to_spherical(xg, yg, zg)

# Calculate the hydrogen orbital wavefunction.
psi_3p = h_orbital(rg,tg,pg,3,1,0)

# The output of h_orbital is a complex type, even though the chosen 
# 3pz orbital is real (imaginary part is zero). Matplotlib can only plot
# real numbers, so we take the real part here.
psi_3p = psi_3p.real

##### Choosing a colormap for plotting

The following cell is a customization of the color map before starting to draw plots in 2D.  Blue and red are used for oppositely signed values. The full list of color maps is available here:

[Colormaps in Matplotlib by name](https://matplotlib.org/stable/tutorials/colors/colormaps.html)

In [ ]:
import matplotlib 
matplotlib.rc('image', cmap='seismic')

##### Plotting the heat map

To make the heat map, we are using [`Axes.pcolormesh`](https://matplotlib.org/api/_as_gen/matplotlib.axes.Axes.pcolormesh.html#matplotlib.axes.Axes.pcolormesh). Its main function signature is `pcolormesh(X,Y,C)` where the arguments are the x values, the y values, and the data to be plotted. By passing the same arguments into `Axes.contour()` we can also create level curves.

Our coordinate and function data are stored in 3D arrays, but `pcolormesh()` can only plot 2D data.  Therefore we need to use indexing to extract a 2D slice out of each 3D array.  This is done using the syntax `xg[:,0,:], zg[:,0,:], psi_3p[:,0,:]` to extract all values in the $x$ and $z$ dimensions but keep only the zeroth element for $y$.  (In this example we only had one value in the $y$ dimension, but reducing the array dimension is still essential.)

We also called `ax.set_aspect('equal', 'box')`.  With this, the size and extent of the horizontal and vertical axes will be set equal and the plot area will be made square.  Otherwise the plot will appear squashed in one direction or another.  This comes in handy when both axes correspond to Cartesian coordinates, and various other situations.

Finally, a color scale is useful for knowing the correspondence between color and function value.  The color scale is the return value of `pcolormesh()`, and it is passed to `Figure.colorbar()` to add a special Axes object containing the color bar.

In [ ]:
# Create a Figure and Axes object
fig, ax = plt.subplots()
# This sets the sets the x- and y- axis limits to be equal
# and makes the plotting area square. 
ax.set_aspect('equal', 'box')
cbar = ax.pcolormesh(xg[:,0,:], zg[:,0,:], psi_3p[:,0,:])
ax.contour(xg[:,0,:], zg[:,0,:], psi_3p[:,0,:], colors='k', 
           linewidths=1, levels=[-0.01, 0.0, 0.01])
fig.colorbar(cbar)

##### Plotting several heat maps in a single figure

Here the calculations and plotting codes are organized into a function, making it easy to create different plots by changing the quantum numbers. Four `Axes` objects are created in a 2x2 grid, making it easy to compare the $xz-$ cross sections of orbitals that differ by only one quantum number; here the quantum numbers are $(4, 0, 0); (4, 1, 0); (4, 2, 0); (4, 3, 0)$.  These correspond to the $4s, 4p_z, 4d_{z^2}$ and $4f_{z^3}$ orbitals.

In these plots we also used the keyword arguments `vmin, vmax` to manually set the color scale (otherwise it would not always be centered around zero). Normally you need to know the range of the values before setting a reasonable color scale. The contour lines are drawn only at $\psi = 0.0$.

In [ ]:
def plot_orbital_xz(x, y, z, n, l, m, fig, ax):
    xg, yg, zg = np.meshgrid(x, y, z, indexing='ij')
    rg, tg, pg = to_spherical(xg, yg, zg)
    psi = h_orbital(rg,tg,pg,n,l,m)
    psi = psi.real
    ax.set_aspect('equal', 'box')
    ax.pcolormesh(xg[:,0,:], zg[:,0,:], psi[:,0,:], vmin=-0.02, vmax=0.02)
    ax.contour(xg[:,0,:], zg[:,0,:], psi[:,0,:], colors='k', 
               linewidths=1, levels=[0.0])
    #fig.colorbar(cbar)
    
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)
x = np.linspace(-10, 10, 101)
y = 0.0
z = np.linspace(-10, 10, 101)
plot_orbital_xz(x, y, z, 4, 0, 0, fig, ax1)
plot_orbital_xz(x, y, z, 4, 1, 0, fig, ax2)
plot_orbital_xz(x, y, z, 4, 2, 0, fig, ax3)
plot_orbital_xz(x, y, z, 4, 3, 0, fig, ax4)

##### Plotting the real and imaginary part

We can also plot orbital cross sections in different planes. Here I've chosen the $xy-$ plane, and chosen the quantum numbers $(3, 2, 2)$. This is the first time we are using nonzero $m$, and these orbitals are complex. The code below plots the real and imaginary part separately. The real part is proportional to $3d_{x^2-y^2}$ and the imaginary part is proportional to $3d_{xy}$. 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2)
x = np.linspace(-10, 10, 101)
y = np.linspace(-10, 10, 101)
z = 0.0
n = 3
l = 2
m = 2

xg, yg, zg = np.meshgrid(x, y, z, indexing='ij')
rg, tg, pg = to_spherical(xg, yg, zg)
psi = h_orbital(rg,tg,pg,n,l,m)

ax1.set_aspect('equal', 'box')
ax1.pcolormesh(xg[:,:,0], yg[:,:,0], psi[:,:,0].real, vmin=-0.02, vmax=0.02)
ax1.contour(xg[:,:,0], yg[:,:,0], psi[:,:,0].real, colors='k', 
            linewidths=1, levels=[0.0])

ax2.set_aspect('equal', 'box')
ax2.pcolormesh(xg[:,:,0], yg[:,:,0], psi[:,:,0].imag, vmin=-0.02, vmax=0.02)
ax2.contour(xg[:,:,0], yg[:,:,0], psi[:,:,0].imag, colors='k', 
            linewidths=1, levels=[0.0])

#### Interactive 3D plots of isosurfaces

Isosurfaces are not a standard feature in current versions of Matplotlib.  Although it is possible to do, one needs to separately calculate the isosurface, and then pass it to Matplotlib as a collection of triangular faces.  On the other hand, the plotly package does support isosurfaces, and the resulting plots are interactive to boot.  Here is an example code for creating a 3D isosurface in Plotly.

Check out the [plotly `Isosurface` documentation](https://plotly.com/python-api-reference/generated/plotly.graph_objects.Isosurface.html). The most important keyword arguments are `x`, `y`, `z`, and `value`, representing the coordinates and function value on a grid of points. Unlike Matplotlib's `pcolormesh`, the coordinate arrays and values are all expected to be 1-D arrays, which can be easily done using the `array.flatten()` method. This is because the `Isosurface` class does not require points to be on a regular 3-D grid.

Other arguments to `Isosurface` include specifying the color scale (`colorscale`), and plotting multiple surfaces by specifying a minimum and maximum value (`isomin` and `isomax`), as well as how many isosurfaces we want (`surface_count`). The `caps` allows the edges of the 3D plot to show the interior of the isosurface (see [Matlab's documentation](https://www.mathworks.com/help/matlab/visualize/isocaps-add-context-to-visualizations.html) for a conceptual explanation, because plotly's documentation is unclear). Here we simplify things by turning them off.

In [ ]:
import plotly.graph_objects as go

x = np.linspace(-15, 15, 51)
y = np.linspace(-15, 15, 51)
z = np.linspace(-15, 15, 51)
n = 3
l = 2
m = 0

xg, yg, zg = np.meshgrid(x, y, z, indexing='ij')
rg, tg, pg = to_spherical(xg, yg, zg)
psi = h_orbital(rg,tg,pg,n,l,m)
psi = psi.real

psimax = psi.max()

iso = go.Isosurface(x=xg.flatten(), y=yg.flatten(), z=zg.flatten(),
                    value=psi.flatten(), colorscale='RdBu',
                    isomin=-.75*psimax, isomax=.75*psimax,
                    surface_count=7,opacity=0.5, caps=dict(x_show=False,y_show=False,z_show=False))

fig= go.Figure(data=iso)

# I thought the default behavior of using scroll to zoom was distracting so I turned it off.
fig.show(config={'scrollZoom': False})

##### Baby steps toward molecular orbitals

The creation of molecular orbitals from linear combinations of atomic orbitals (LCAO-MO) is a foundation of modern quantum chemistry.  At its core is a very simple concept: MOs are formed by taking linear combinations of AOs - essentially hydrogen orbitals - centered on different atoms in the molecule.  

The math required to describe a MO is a simple extension of the hydrogen orbitals.  We need a way to move the AO center to different locations, and this could simply be done by adding an `origin` keyword argument to the `to_spherical()` function that computes the spherical coordinates. 

In [ ]:
def to_spherical(x, y, z, origin=np.array([0, 0, 0])):
    """Converts the Cartesian coordinates x,y,z to r,theta,phi

    This function expects x, y, and z to be 3D arrays,
    such as those generated by np.meshgrid. The output arrays are
    always dense 3D grids, and correspond to r, theta, and phi

    Parameters
    ----------
    x, y, z : np.ndarray
        Cartesian x / y / z values, in a.u.

    Returns
    -------
    tuple of 3x np.ndarrays
        3D grid of r, theta, phi values
    """
    dx = x - origin[0]
    dy = y - origin[1]
    dz = z - origin[2]
    
    r = ( dx**2 + dy**2 + dz**2 )**0.5
    theta = np.arctan2((dx**2 + dy**2)**0.5, dz)
    phi = np.arctan2(dy, dx)

    return r, theta, phi

# Convenience function so we don't have to enter 
# the keyword arguments repeatedly. Setting an odd number 
# of surfaces will result in showing the nodal surfaces (isovalue 0).
def plotly_isosurface(xg, yg, zg, psi):
    psimax = psi.max()
    return go.Isosurface(x=xg.flatten(), y=yg.flatten(), z=zg.flatten(),
                         value=psi.flatten(), colorscale='RdBu',
                         isomin=-.75*psimax, isomax=.75*psimax,
                         surface_count=7,opacity=0.5, 
                         caps=dict(x_show=False,y_show=False,z_show=False))

Here's an orbital plot with the origin shifted to $(0, 0, -5)$ on the $z-$axis. Again we are taking the real part of the orbital, which is proportional to the $3d_{yz}$ orbital in this case. More rigorously, the $3d_{xz}$ and $3d_{yz}$ orbitals can be obtained by taking linear combinations of the complex $3d$ orbitals with $m=-1$ and $m=+1$.

In [ ]:
rg, tg, pg = to_spherical(xg, yg, zg, origin=np.array([0, 0, -5]))
psi = h_orbital(rg,tg,pg,3,2,1)
psi = psi.real

iso = plotly_isosurface(xg, yg, zg, psi)

fig= go.Figure(data=iso)
fig.show(config={'scrollZoom': False})

By adding two orbitals with different centers in an in-phase way, we can make a visualization of a bonding molecular orbital.  In actual QM calculations, the atomic orbital sizes would be affected by the nuclear charge and core electron shielding, and sums of Gaussian functions are typically used to approximate the exponential decay of the orbital with increasing $r$.

In [ ]:
rg1, tg1, pg1 = to_spherical(xg, yg, zg, origin=np.array([0, 0, -5]))
psi1 = h_orbital(rg1,tg1,pg1,3,2,1)
psi1 = psi1.real

rg2, tg2, pg2 = to_spherical(xg, yg, zg, origin=np.array([0, 0, 6]))
psi2 = h_orbital(rg2,tg2,pg2,2,1,1)
psi2 = psi2.real * 0.5 # You can tune this constant to see how it affects the orbital shapes.

psi = psi1 + psi2

iso = plotly_isosurface(xg, yg, zg, psi)

fig= go.Figure(data=iso)
fig.show(config={'scrollZoom': False})

Does plotly support having multiple subplots in the same figure? Yes, it does. Here we draw the two AOs that contribute to form the bonding and antibonding MO.

In [ ]:
from plotly.subplots import make_subplots
# Initialize figure with 4 3D subplots
fig = make_subplots(
    rows=1, cols=4,
    specs=[[{'type': 'surface'}, {'type': 'surface'},
            {'type': 'surface'}, {'type': 'surface'}]])

rg1, tg1, pg1 = to_spherical(xg, yg, zg, origin=np.array([0, 0, -4]))
psi1 = h_orbital(rg1,tg1,pg1,3,2,1)
psi1 = psi1.real

rg2, tg2, pg2 = to_spherical(xg, yg, zg, origin=np.array([0, 0, 4]))
psi2 = h_orbital(rg2,tg2,pg2,2,1,1)
psi2 = psi2.real * 0.5

psi_bond = psi1 + psi2
psi_anti = psi1 - psi2

iso1 = plotly_isosurface(xg, yg, zg, psi1)
iso2 = plotly_isosurface(xg, yg, zg, psi2)
iso_bond = plotly_isosurface(xg, yg, zg, psi_bond)
iso_anti = plotly_isosurface(xg, yg, zg, psi_anti)

# adding surfaces to subplots.
fig.add_trace(iso1, row=1, col=1)
fig.add_trace(iso2, row=1, col=2)
fig.add_trace(iso_bond, row=1, col=3)
fig.add_trace(iso_anti, row=1, col=4)

There are many possible ways to extend this code. You can write a function that makes it convenient to draw multiple orbitals, each having a different set of three quantum numbers and a different origin. Ultimately you could make visualizations of the MO diagrams that you see in organic and inorganic chemistry textbooks. The possibilities are endless!